In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [18]:
username='natgeo'
browser = webdriver.Chrome('/Users/matt/Downloads/chromedriver')
browser.get('https://www.instagram.com/'+username+'/')

In [19]:
links=[]

def scrape(start, end):
    print('scrolling: {}-{}'.format(start, end))
    Pagelength = browser.execute_script("window.scrollTo({}, {});".format(start, end))
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            links.append('https://www.instagram.com'+link.get('href'))
    
    time.sleep(5)
    if (end < 500000):
        scrape(start + 3000, end + 3000)

In [20]:
scrape(0, 6000)

scrolling: 0-6000
scrolling: 3000-9000
scrolling: 6000-12000
scrolling: 9000-15000
scrolling: 12000-18000
scrolling: 15000-21000
scrolling: 18000-24000
scrolling: 21000-27000
scrolling: 24000-30000
scrolling: 27000-33000
scrolling: 30000-36000
scrolling: 33000-39000
scrolling: 36000-42000
scrolling: 39000-45000
scrolling: 42000-48000
scrolling: 45000-51000
scrolling: 48000-54000
scrolling: 51000-57000
scrolling: 54000-60000
scrolling: 57000-63000
scrolling: 60000-66000
scrolling: 63000-69000
scrolling: 66000-72000
scrolling: 69000-75000
scrolling: 72000-78000
scrolling: 75000-81000
scrolling: 78000-84000
scrolling: 81000-87000
scrolling: 84000-90000
scrolling: 87000-93000
scrolling: 90000-96000
scrolling: 93000-99000
scrolling: 96000-102000
scrolling: 99000-105000
scrolling: 102000-108000
scrolling: 105000-111000
scrolling: 108000-114000
scrolling: 111000-117000
scrolling: 114000-120000
scrolling: 117000-123000
scrolling: 120000-126000
scrolling: 123000-129000
scrolling: 126000-132000


In [21]:
len(links)

6378

In [24]:
result=pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
        x.columns =  x.columns.str.replace("shortcode_media.", "")
        result=result.append(x)       
    except:
        np.nan

result.index = range(len(result.index))

In [25]:
len(result)

935

In [26]:
properties = ['__typename', 'display_url', 'edge_media_to_caption.edges', 'edge_media_preview_like.count', 'edge_media_preview_comment.count']
columns = ['type', 'media_url', 'caption', 'likes', 'comments']

output = result.copy()
output = output[properties]
output.columns = columns
output.caption = output.caption.apply(lambda s: str(s)
  .replace("[{'node': {'text': ", '')
  .replace("'", '')
  .replace("\"", ''))

print('scraped {} posts'.format(len(output)))
output.head(10)

scraped 935 posts


,type,media_url,caption,likes,comments
0,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,"Photo by @jasperdoest | Penguins, like other b...",123736,697
1,GraphSidecar,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photos by Pete McBride @pedromcbride | Source ...,359268,761
2,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by William Albert Allard @williamalberta...,176725,343
3,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Katie Orlinsky @katieorlinsky | A you...,205613,859
4,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Trevor Frost @tbfrost | Snakes in car...,188741,920
5,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Michael Melford @michaelmelford | An ...,293077,589
6,GraphVideo,https://scontent-ort2-1.cdninstagram.com/v/t51...,Video by Joel Sartore @joelsartore | Pemba Isl...,114722,647
7,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by @gerdludwig | Wearing the local costu...,292550,1636
8,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Saumya Khandelwal @khandelwal_saumya ...,136274,294
9,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Keith Ladzinski @ladzinski | In searc...,751550,2898


In [31]:
output = output.drop_duplicates()
images = output[output.type == 'GraphImage']
print('Images length: {}'.format(len(images)))
images.to_csv('output.csv', columns=columns, index=False)

Images length: 288


In [32]:
df = pd.read_csv('output.csv')

In [33]:
test = df.copy()
test = test.drop_duplicates()

In [38]:
test.head()

,type,media_url,caption,likes,comments
0,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,"Photo by @jasperdoest | Penguins, like other b...",123736,697
1,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by William Albert Allard @williamalberta...,176725,343
2,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Katie Orlinsky @katieorlinsky | A you...,205613,859
3,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Trevor Frost @tbfrost | Snakes in car...,188741,920
4,GraphImage,https://scontent-ort2-1.cdninstagram.com/v/t51...,Photo by Michael Melford @michaelmelford | An ...,293077,589
